<a href="https://colab.research.google.com/github/analyticsariel/projects/blob/master/Rentometer_RentEstimates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rentometer Rental Data

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com |
| External References   | API |
| Input Datasets        | Source name |
| Output Datasets       | Source name |
| Input Data Source     | Pandas DataFrame |
| Output Data Source    | Pandas DataFrame |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 2nd Jan 2021 | Ariel Herrera | Create notebook. |

## Getting Started
1. Copy this notebook -> File -> Save a Copy in Drive
2. Directions

## Useful Resources
- [Google Colab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)
- [Rentometer API](https://www.rentometer.com/rentometer-api-release-notes)
- [How to convert JSON into a Pandas DataFrame](https://towardsdatascience.com/how-to-convert-json-into-a-pandas-dataframe-100b2ae1e0d8)
- [Convert CURL request to Python](https://curlconverter.com/#)

## <font color="blue">Install Packages</font>

## <font color="blue">Imports</font>

In [ ]:
from google.colab import drive, files # google colab specific
import requests
import pandas as pd
import os
import warnings

## <font color="blue">Functions</font>

In [ ]:
def get_rentometer_summary(api_key,
                           address=None,
                           latitude=None,
                           longitude=None,
                           bedrooms=None,
                           baths=None,
                           building_type=None,
                           look_back_days=None):
  # check if required parameters are present
  if (address == None) and (latitude == None or longitude == None):
    return "Please enter <address> or <latitude> AND <longitude> parameters."

  # variables
  base_url = 'https://www.rentometer.com/api/v1/summary'
  # get bedrooms
  if bedrooms == None:
    pass
  elif int(bedrooms) > 4:
    bedrooms = '4'
  else:
    bedrooms = str(bedrooms)
  # get baths
  if baths == None:
    pass
  elif int(baths) > 1:
    baths = '1.5+'
  else:
    baths = '1'
  # get look back days
  if look_back_days == None:
    pass
  elif int(look_back_days) > 1460:
    look_back_days = 1460
  elif int(look_back_days) < 90:
    look_back_days = 90
  else:
    pass

  params = (
      ('api_key', api_key),
      ('address', address),
      ('latitude', latitude),
      ('longitude', longitude),
      ('bedrooms', bedrooms),
      ('baths', baths),
      ('building_type', building_type), # valid values: [apartments, house]
      ('look_back_days', look_back_days)
  )

  response = requests.get(base_url, params=params)
  return response

## <font color="blue">Locals & Constants</font>

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# set working directory
os.chdir('/content/drive/MyDrive/Colab Data/')
dir = os.getcwd()
print('Current working directory:', dir)

# remove pandas warnings
warnings.filterwarnings('ignore')

Current working directory: /content/drive/MyDrive/Colab Data


## <font color="blue">Data</font>

In [ ]:
# read in api keys
df_api_keys = pd.read_csv(dir + '/input/api_keys.csv')
# extract api key
rentometer_api_key = df_api_keys.loc[df_api_keys['API'] == 'rentometer']['KEY'].iloc[0]

## <font color="blue">Transformations</font>

### <font color="Green">Sample API Request - Single Address</font>

In [ ]:
# sample API get request
base_url = 'https://www.rentometer.com/api/v1/summary'

params = (
    ('api_key', rentometer_api_key),
    ('address', '15 broad st., boston, ma'),
    ('bedrooms', '2'),
    ('baths', '1.5+'),
    ('building_type', 'house'),
)

response = requests.get(base_url, params=params)
print(response.text)

{"address":"15 Broad Street, Boston, Massachusetts 02109","latitude":"42.358729","longitude":"-71.054801","bedrooms":2,"baths":"1.5+","building_type":"House","look_back_days":365,"mean":3444,"median":3200,"min":2900,"max":4700,"percentile_25":3034,"percentile_75":3855,"std_dev":609,"samples":7,"radius_miles":2.0,"quickview_url":"https://www.rentometer.com/analysis/2-bed/15-broad-street-boston-massachusetts-02109/oAGyreNxafo/quickview","credits_remaining":490,"token":"oAGyreNxafo","links":[{"rel":"request pro report","href":"/api/v1/request_pro_report?api_key=831TfCG3Hp2S58F3Rq2d9w&token=oAGyreNxafo"}]}


In [ ]:
# transform to json response
response_json = response.json()
response_json

{'address': '15 Broad Street, Boston, Massachusetts 02109',
 'baths': '1.5+',
 'bedrooms': 2,
 'building_type': 'House',
 'credits_remaining': 490,
 'latitude': '42.358729',
 'links': [{'href': '/api/v1/request_pro_report?api_key=831TfCG3Hp2S58F3Rq2d9w&token=oAGyreNxafo',
   'rel': 'request pro report'}],
 'longitude': '-71.054801',
 'look_back_days': 365,
 'max': 4700,
 'mean': 3444,
 'median': 3200,
 'min': 2900,
 'percentile_25': 3034,
 'percentile_75': 3855,
 'quickview_url': 'https://www.rentometer.com/analysis/2-bed/15-broad-street-boston-massachusetts-02109/oAGyreNxafo/quickview',
 'radius_miles': 2.0,
 'samples': 7,
 'std_dev': 609,
 'token': 'oAGyreNxafo'}

In [ ]:
# normalize into a tabular format
df = pd.json_normalize(
    data=response_json
    )
df.head()

,address,latitude,longitude,bedrooms,baths,building_type,look_back_days,mean,median,min,max,percentile_25,percentile_75,std_dev,samples,radius_miles,quickview_url,credits_remaining,token,links
0,"15 Broad Street, Boston, Massachusetts 02109",42.358729,-71.054801,2,1.5+,House,365,3444,3200,2900,4700,3034,3855,609,7,2.0,https://www.rentometer.com/analysis/2-bed/15-b...,490,oAGyreNxafo,"[{'rel': 'request pro report', 'href': '/api/v..."


In [ ]:
# download
df.to_csv('rentometer_rents_single_address.csv', index=False)
files.download('rentometer_rents_single_address.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### <font color="Green">Function API Request - Multiple Addresses</font>

In [ ]:
# create a dataframe
d = {'address': ['1921 E Hanna Ave, Tampa, FL 33610', '2516 Mabry St, Tampa, FL 33618'], 
     'bedrooms': [3, 4],
     'bathrooms': [2, 2.5]}
df_addresses = pd.DataFrame(data=d)
df_addresses

,address,bedrooms,bathrooms
0,"1921 E Hanna Ave, Tampa, FL 33610",3,2.0
1,"2516 Mabry St, Tampa, FL 33618",4,2.5


In [ ]:
# option 1 - for loop

response_list = []
# iterate through rows in the dataframe (table)
for index, row in df_addresses.iterrows():
  # get parameters
  address = row['address']
  bds = row['bedrooms']
  baths = row['bathrooms']
  # get api response
  print('Getting data for address: {0}'.format(address))
  response = get_rentometer_summary(api_key=rentometer_api_key,
                           address=address,
                           bedrooms=bds,
                           baths=baths)
  response_list.append(response)

Getting data for address: 1921 E Hanna Ave, Tampa, FL 33610
Getting data for address: 2516 Mabry St, Tampa, FL 33618


In [ ]:
# view a single response
response_list[0].text

'{"address":"1921 East Hanna Avenue, Tampa, Florida 33610","latitude":"28.003158","longitude":"-82.437086","bedrooms":3,"baths":"1.5+","building_type":"Any","look_back_days":365,"mean":1927,"median":1895,"min":800,"max":2800,"percentile_25":1577,"percentile_75":2277,"std_dev":519,"samples":21,"radius_miles":1.0,"quickview_url":"https://www.rentometer.com/analysis/3-bed/1921-east-hanna-avenue-tampa-florida-33610/41hPBcAge6c/quickview","credits_remaining":489,"token":"41hPBcAge6c","links":[{"rel":"request pro report","href":"/api/v1/request_pro_report?api_key=831TfCG3Hp2S58F3Rq2d9w&token=41hPBcAge6c"}]}'

In [ ]:
# get response values into lists
rent_mean = [resp.json()['mean'] for resp in response_list]
rent_median = [resp.json()['median'] for resp in response_list]
rent_max = [resp.json()['max'] for resp in response_list]
rent_min = [resp.json()['min'] for resp in response_list]

In [ ]:
# add new columns to dataframe
df_addresses_rent = df_addresses.copy()
df_addresses_rent['rent_mean']= rent_mean
df_addresses_rent['rent_median']= rent_median
df_addresses_rent['rent_max']= rent_max
df_addresses_rent['rent_min']= rent_min
df_addresses_rent

,address,bedrooms,bathrooms,rent_mean,rent_median,rent_max,rent_min
0,"1921 E Hanna Ave, Tampa, FL 33610",3,2.0,1927,1895,2800,800
1,"2516 Mabry St, Tampa, FL 33618",4,2.5,1706,1575,2700,950


## <font color="blue">Output</font>

In [ ]:
# download
df_addresses_rent.to_csv('rentometer_rents.csv', index=False)
files.download('rentometer_rents.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# End Notebook